# 스케일링(Scaling)

* 다음의 내용을 설명합니다.
    * 표준화와 정규화 차이
    * 스케일링 한 데이터와 안한 데이터 knn 성능 차이

## 1.환경준비

### (1) 라이브러리 로딩

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.neighbors import KNeighborsRegressor

### (2) 데이터로딩

In [ ]:
path = 'https://raw.githubusercontent.com/DA4BAM/dataset/master/boston.csv'
data = pd.read_csv(path)
data.head()

|	변수	|	설명	|
|----|----|
|	medv	|	타운별 집값(중위수), **Target**	|
|	crim	|	범죄율	|
|	zn	|	25,000 평방피트를 초과 거주지역 비율	|
|	indus	|	비소매상업지역 면적 비율	|
|	chas	|	찰스강변 위치(범주 : 강변1, 아니면 0)	|
|	nox	|	일산화질소 농도	|
|	rm	|	주택당 방 수	|
|	age	|	1940년 이전에 건축된 주택의 비율	|
|	dis	|	직업센터의 거리	|
|	rad	|	방사형 고속도로까지의 거리	|
|	tax	|	재산세율	|
|	ptratio	|	학생/교사 비율	|
|	black	|	인구 중 흑인 비율	|
|	lstat	|	인구 중 하위 계층 비율	|


## 2.데이터 전처리

In [ ]:
target = 'medv'
x = data.drop(target, axis = 1)
y = data.loc[:, target]

x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=.2
                                                  , random_state = 20)

## 3.스케일링

### (1) 정규화(Normalization)

In [ ]:
scaler = MinMaxScaler()
x_train_n = scaler.fit_transform(x_train)
x_val_n = scaler.transform(x_val)

# 데이터프레임으로 변환하는 이유는 .describe() 메소드를 사용하기 위함 입니다.
x_train_n = pd.DataFrame(x_train_n, columns = list(x_train))
x_val_n = pd.DataFrame(x_val_n, columns = list(x_val))

### (2) 표준화(Standardization)

In [ ]:
scaler = StandardScaler()
x_train_s = scaler.fit_transform(x_train)
x_val_s = scaler.transform(x_val)

x_train_s = pd.DataFrame(x_train_s, columns = list(x_train))
x_val_s = pd.DataFrame(x_val_s, columns = list(x_val))

## 4.표준화 vs 정규화

### (1) 원본과 스케일링 된 데이터 비교하기
* 위 결과에서 특별히 crim, tax, nox만 살펴보겠습니다.

In [ ]:
features = ['crim', 'tax', 'nox']

* 원본 데이터

In [ ]:
# 원본데이터에서 crim, tax, nox만 뽑아서 기초통계량을 살펴보겠습니다.
# .T는 transpose 메소드로, 행과 열의 위치를 바꾸는 일을 합니다. 
x_train[features].describe().T

* 정규화 데이터 기초통계량

세 변수의 min은 0, max은 1로 변환되었습니다.

In [ ]:
x_train_n[features].describe().T

* 표준화 데이터 기초통계량

평균은 거의 0, 표준편차는 거의 1 입니다.

In [ ]:
x_train_s[features].describe().T

* 스케일링은 분포의 모양 자체를 바꾸지는 않습니다.

* crim

In [ ]:
var = 'crim'

plt.figure(figsize = (15,5))
plt.subplot(1,3,1)
plt.title('Original')
sns.kdeplot(x_train[var])
plt.grid()

plt.subplot(1,3,2)
plt.title('Normalization')
sns.kdeplot(x_train_n[var])
plt.grid()

plt.subplot(1,3,3)
plt.title('Standardization')
sns.kdeplot(x_train_s[var])
plt.grid()

plt.tight_layout()
plt.show()

* tax

In [ ]:
var = 'tax'

plt.figure(figsize = (15,5))
plt.subplot(1,3,1)
plt.title('Original')
sns.kdeplot(x_train[var])
plt.grid()

plt.subplot(1,3,2)
plt.title('Normalization')
sns.kdeplot(x_train_n[var])
plt.grid()

plt.subplot(1,3,3)
plt.title('Standardization')
sns.kdeplot(x_train_s[var])
plt.grid()

plt.tight_layout()
plt.show()

* nox

In [ ]:
var = 'nox'

plt.figure(figsize = (15,5))
plt.subplot(1,3,1)
plt.title('Original')
sns.kdeplot(x_train[var])
plt.grid()

plt.subplot(1,3,2)
plt.title('Normalization')
sns.kdeplot(x_train_n[var])
plt.grid()

plt.subplot(1,3,3)
plt.title('Standardization')
sns.kdeplot(x_train_s[var])
plt.grid()

plt.tight_layout()
plt.show()


### (2) 모델링 성능 비교

* 원본, 정규화, 표준화 데이터셋에 대해서
* k값을 증가시켜가며 모델생성, 예측, MAE 평가를 수행한 후
* 이를 그래프로 그려 보았습니다.

In [ ]:
k_vals = list(range(1,50))
r1, r2, r3 = [],[],[]

for k in k_vals : 
    m1 = KNeighborsRegressor(n_neighbors = k)
    m2 = KNeighborsRegressor(n_neighbors = k)
    m3 = KNeighborsRegressor(n_neighbors = k)

    m1.fit(x_train, y_train)
    m2.fit(x_train_n, y_train)
    m3.fit(x_train_s, y_train)

    p1 = m1.predict(x_val)
    p2 = m2.predict(x_val_n)
    p3 = m3.predict(x_val_s)

    r1.append(mean_absolute_error(y_val, p1))
    r2.append(mean_absolute_error(y_val, p2))
    r3.append(mean_absolute_error(y_val, p3))


In [ ]:
plt.figure(figsize = (10,6))
plt.plot(k_vals, r1, marker = '.', label = 'Original')
plt.plot(k_vals, r2, marker = '.', label = 'Normalization')
plt.plot(k_vals, r3, marker = '.', label = 'Standardization')

plt.xlabel('k(n_neighbors)')
plt.ylabel('mean absolute error')
plt.grid()
plt.legend()
plt.show()

## 5.실습
* 원본데이터와 스케일링 된 두 데이터에 대해서 SVM으로 모델링을 수행하고 성능을 비교해 봅시다.
* 4 - (2) 코드와 같이
    * C 를 조금씩 조절하며 성능비교를 합시다.
        * 범위 : 0.01 ~ 10

In [ ]:
from sklearn.svm import SVR

In [ ]:
np.linspace(0.01, 10, 100)

In [ ]:
c_values = np.linspace(0.01, 10, 100)
r1, r2, r3 = [],[],[]

for c in c_values : 
    m1 = SVR(C = c)
    m2 = SVR(C = c)
    m3 = SVR(C = c)

    m1.fit(x_train, y_train)
    m2.fit(x_train_n, y_train)
    m3.fit(x_train_s, y_train)

    p1 = m1.predict(x_val)
    p2 = m2.predict(x_val_n)
    p3 = m3.predict(x_val_s)

    r1.append(mean_absolute_error(y_val, p1))
    r2.append(mean_absolute_error(y_val, p2))
    r3.append(mean_absolute_error(y_val, p3))


In [ ]:
plt.figure(figsize = (10,6))
plt.plot(c_values, r1, marker = '.', label = 'Original')
plt.plot(c_values, r2, marker = '.', label = 'Normalization')
plt.plot(c_values, r3, marker = '.', label = 'Standardization')

plt.xlabel('k(n_neighbors)')
plt.ylabel('mean absolute error')
plt.grid()
plt.legend()
plt.show()